In [ ]:
import anndata
import scvelo as scv
import pandas as pd
import numpy as np
import matplotlib as plt
import scanpy as sc
import cellrank as cr

scv.logging.print_version()
scv.settings.presenter_view = True
scv.settings.set_figure_params('scvelo')
scv.settings.verbosity = 3
cr.settings.verbosity = 2

In [ ]:
# import output loom files from velocyto
s1 = anndata.read_loom("1551_1/possorted_genome_bam_NZ7WX.loom")
s2 = anndata.read_loom("1551_3/possorted_genome_bam_MF8C0.loom")
s3 = anndata.read_loom("1551_5/possorted_genome_bam_54MM0.loom")
s4 = anndata.read_loom("1551_7/possorted_genome_bam_BUHZG.loom")
s5 = anndata.read_loom("Blood_d10/H1/possorted_genome_bam_HGP3U.loom")
s6 = anndata.read_loom("Blood_d10/H2/possorted_genome_bam_AWTRY.loom")
s7 = anndata.read_loom("Blood_d10/H3/possorted_genome_bam_YNDJD.loom")
s8 = anndata.read_loom("Blood_d20/H1/possorted_genome_bam_EUWL2.loom")
s9 = anndata.read_loom("Blood_d20/H2/possorted_genome_bam_YO0JD.loom")
s10 = anndata.read_loom("Blood_d20/H3/possorted_genome_bam_P3A1A.loom")
s11 = anndata.read_loom("Blood_d30/H1/possorted_genome_bam_YQ8ZV.loom")
s12 = anndata.read_loom("Blood_d30/H2/possorted_genome_bam_TE8TY.loom")
s13 = anndata.read_loom("Blood_d30/H3/possorted_genome_bam_43CNI.loom")
s14 = anndata.read_loom("Healthy_Blood/H1/possorted_genome_bam_Q1PXG.loom")
s15 = anndata.read_loom("Healthy_Blood/H2/possorted_genome_bam_WXOUL.loom")
s16 = anndata.read_loom("Healthy_Blood/H3/possorted_genome_bam_566KS.loom")

In [ ]:
s1.var_names_make_unique()
s2.var_names_make_unique()
s3.var_names_make_unique()
s4.var_names_make_unique()
s5.var_names_make_unique()
s6.var_names_make_unique()
s7.var_names_make_unique()
s8.var_names_make_unique()
s9.var_names_make_unique()
s10.var_names_make_unique()
s11.var_names_make_unique()
s12.var_names_make_unique()
s13.var_names_make_unique()
s14.var_names_make_unique()
s15.var_names_make_unique()
s16.var_names_make_unique()

In [ ]:
cell_names=s16.obs.index
cell_names_mod = [sub.replace('possorted_genome_bam_566KS:', 'Healthy_Blood_3_') for sub in cell_names]
cell_names_mod = [sub.replace('x', '') for sub in cell_names_mod]
s16.obs.index = cell_names_mod

In [ ]:
tsne = pd.read_csv('tmp/tsne.csv', index_col=0) # load tsne embedding computed with Palantir
tsne

In [ ]:
cluster = pd.read_csv('annotated_clusters_subset.csv', index_col=0) # load cell annotation
cluster = cluster.rename(columns={'x':'clusters_refined'})
cluster

In [ ]:
tissue = pd.read_csv('tissue_annotation.csv', index_col=0) # load tissue annotation
tissue

In [ ]:
merge = s1.concatenate(s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16)
merge

In [ ]:
cell_names=merge.obs.index
cell_names_mod = [sub.replace('-0', '-1') for sub in cell_names]
cell_names_mod = [sub.replace('-2', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-3', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-4', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-5', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-6', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-7', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-8', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-9', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-10', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-11', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-12', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-13', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-14', '-1') for sub in cell_names_mod]
cell_names_mod = [sub.replace('-15', '-1') for sub in cell_names_mod]
merge.obs.index = cell_names_mod
merge.obs.index

In [ ]:
ad=merge[np.isin(merge.obs.index, tsne.index)]
tsne = tsne.loc[ad.obs.index,:]
cluster = cluster.loc[ad.obs.index,:]
tissue = tissue.loc[ad.obs.index,:]
ad.obs['cluster'] = cluster.values
ad.obs['tissue'] = tissue.values
ad.obsm['X_tsne'] = tsne.values
ad

In [ ]:
np.allclose(np.ravel(ad.X[:5].data) % 1, 0, atol=1e-3)

In [ ]:
scv.pl.proportions(ad,groupby='cluster')

In [ ]:
scv.pp.filter_and_normalize(ad, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(ad, n_pcs=25, n_neighbors=30)
scv.tl.recover_dynamics(ad)
scv.tl.velocity(ad, mode = 'dynamical')
scv.tl.velocity_graph(ad)

In [ ]:
scv.pl.velocity_embedding_stream(ad, basis='tsne', color='cluster',save='Velocity_MonoMacro_dynamical_TSNEMNN.svg',palette=["#ede445ff",'#43e0efff',"#78f785ff","#e378d0ff","#ff5c67ff","#f7903bff","#a179e2ff","#5d97eaff"],legend_loc='none')
scv.pl.velocity_embedding_stream(ad, basis='tsne', color='tissue',save='Tissue_Velocity_MonoMacro_dynamical_TSNEMNN.svg',palette=["#ede445ff",'#ff5c67ff',"#5d97eaff"],legend_loc='none')

In [ ]:
from cellrank.tl.kernels import VelocityKernel
from cellrank.tl.kernels import ConnectivityKernel
vk = VelocityKernel(ad)
vk.compute_transition_matrix()
ck = ConnectivityKernel(ad).compute_transition_matrix()
combined_kernel = 0.8 * vk + 0.2 * ck
print(combined_kernel)

In [ ]:
from cellrank.tl.estimators import GPCCA
g = GPCCA(combined_kernel)
print(g)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum()
g.plot_schur(basis='tsne',use=10, ncols=3)

In [ ]:
g.compute_macrostates(n_states=6, cluster_key="cluster")
g.plot_coarse_T(text_kwargs={"fontsize": 10})
g.macrostates_memberships

In [ ]:
g.plot_macrostates(basis='tsne',discrete=True, save='Macrostates_TSNE.pdf')
g.compute_terminal_states(method="top_n", n_states=3)
g.plot_terminal_states(basis='tsne',same_plot=True,discrete=True, save='All_Terminal_states_TSNE.pdf')

In [ ]:
g._set_initial_states_from_macrostates(["Classical_Mono_1"])
g.compute_absorption_probabilities()
g.absorption_probabilities

In [ ]:
Il1b_abs = g.absorption_probabilities['Il1b+_TAMs']
Il1b_abs
ad.obs['Il1b_absorption_prob'] = Il1b_abs
Il1b_abs.to_csv('Il1b_absorption_probabilities.csv')

In [ ]:
Il1b_drivers = g.compute_lineage_drivers(lineages="Il1b+_TAMs", cluster_key = 'cluster', clusters = 'Classical_Mono', return_drivers=True, method='perm_test',n_perms=1000,seed=123)
Il1b_drivers
Il1b_drivers.sort_values(by="Il1b+_TAMs corr", ascending=False)
Il1b_drivers.to_csv('TAM_Il1b_lineage_drivers.csv')
g.plot_lineage_drivers("Il1b+_TAMs", basis='tsne', n_genes=5, save='Top5_genedrivers_Cl.Mono_to_Il1b_TSNE.pdf')